In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv("../data/raw/liver_cancer_prediction.csv")

: 

## EDA

In [ ]:
data.head()

: 

In [ ]:
print("Dataset Shape:", data.shape)

: 

In [ ]:
data.info()

: 

In [ ]:
# Check for missing values
print("Missing Values:\n", data.isnull().sum())

: 

The dataset has no missing values across any of the 25 variables. This completeness is beneficial for modeling as it eliminates the risk of bias introduced by missing data handling methods such as mean imputation or predictive modeling.

In [ ]:
# Check for duplicate rows
duplicates = data.duplicated().sum()
print("Number of duplicate rows:", duplicates)

: 

In [ ]:
# Get summary statistics for numerical features
data.describe()

: 

In [ ]:
# Check unique values and percentage distribution in categorical columns
categorical_cols = data.select_dtypes(include=['object']).columns

for col in categorical_cols:
    value_counts = data[col].value_counts()
    percentage = (value_counts / len(data)) * 100  # Calculate percentage
    
    print(f"\n{col} - Unique Values and Distribution:\n")
    print(pd.DataFrame({'Count': value_counts, 'Percentage': percentage.round(2)}))

: 

The dataset paints a pretty interesting picture of liver cancer risk factors and healthcare access across the world. It covers a wide range of countries, with Italy, Egypt, DR Congo, and France each making up about 3.4% of the dataset. Regionally, Sub-Saharan Africa and Europe take the lead, each accounting for about 20%, while Southeast Asia (16.6%) and South Asia (10%) also have notable representation. On the other hand, North America and Latin America have the smallest share, each contributing just over 3%. This suggests that liver cancer is being studied more extensively in regions where the disease may be more prevalent or where risk factors are more pronounced.

When it comes to demographics, men dominate the dataset (70%), which might indicate that they’re at higher risk for liver cancer compared to women. As for lifestyle factors, smoking and alcohol consumption are pretty evenly spread out, with nearly half of the individuals being smokers and alcohol use divided fairly equally across low, moderate, and high levels. Health conditions like obesity and diabetes are also major factors, with about a quarter of the population being classified as obese and nearly 20% diagnosed with diabetes—both known risk factors for liver disease.

A big concern here is healthcare access, or rather, the lack of it. About 60% of the population doesn’t have access to screening, and half of them can’t get proper treatment. Even more alarming, 80% of cases don’t have access to liver transplants, which could be a lifesaving option for some. The dataset also shows a fairly balanced ethnic representation, with African, Hispanic, Asian, Caucasian, and mixed populations each making up around 20%. In terms of preventive care, most people fall into the “moderate” care category (40%), but there’s still a significant portion receiving either poor (30%) or good (30%) care. Lastly, when looking at liver cancer predictions, about 75% of cases are predicted as low risk, while 25% are classified as high risk. That’s still a sizable number of people who might need intervention, highlighting the importance of early detection and treatment options.

There is an imbalance in the dataset. 
Majority of dependent variable values are "No". 
Similarily, some independent variables are imbalanced. Refer to outputs below

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Count plot for target variable
plt.figure(figsize=(6, 4))
sns.countplot(x=data["Prediction"])
plt.title("Distribution of Liver Cancer Prediction (Target Variable)")
plt.xlabel("Prediction (Yes/No)")
plt.ylabel("Count")
plt.show()

: 

In [ ]:
# List of numeric columns
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns

# Histograms for numerical features
data[numerical_cols].hist(figsize=(15, 12), bins=20, color='skyblue', edgecolor='black')
plt.suptitle("Distribution of Numerical Features", fontsize=16)
plt.show()

: 

## Feature Engineering

In [ ]:
categorical_cols = ['Country', 'Region', 'Gender', 'Alcohol_Consumption', 'Smoking_Status',
                'Hepatitis_B_Status', 'Hepatitis_C_Status', 'Obesity', 'Diabetes', 
                'Rural_or_Urban', 'Seafood_Consumption', 'Herbal_Medicine_Use',
                'Healthcare_Access', 'Screening_Availability', 'Treatment_Availability',
                'Liver_Transplant_Access', 'Ethnicity', 'Preventive_Care']

for col in categorical_cols:
    plt.figure(figsize=(10, 4))
    order = data[col].value_counts().index  # Sorting by frequency
    sns.countplot(x=col, data=data, order=order)
    plt.title(f'Count Plot of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation=45)  # Rotate x-labels if needed for clarity
    plt.show()
    
    # Print frequency counts
    print(f'\nValue counts for {col}:')
    print(data[col].value_counts())

: 

In [ ]:
# Box plot for each numerical feature to detect outliers
# Loop through numerical columns and calculate outliers using the IQR method
for col in numerical_cols:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify the outliers
    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    
    print(f'{col}:')
    print(f'  Lower bound: {lower_bound}')
    print(f'  Upper bound: {upper_bound}')
    print(f'  Number of outliers: {len(outliers)}')
    print()

: 

The IQR-based outlier detection results indicate that none of the numerical features—Population, Incidence_Rate, Mortality_Rate, Age, Survival_Rate, and Cost_of_Treatment—have any values falling outside the computed lower and upper bounds. Although the calculated lower bounds for several features are negative (for instance, Population at –746,981,231.75 and Survival_Rate at –29.43), this outcome is a byproduct of the IQR method and does not necessarily imply that the actual data contains negative values. The absence of detected outliers suggests that the data points for these features are relatively concentrated within a consistent range, which bodes well for subsequent analyses. However, it is important to verify that these negative thresholds are appropriate given the context of each variable—especially for features that should logically only have non-negative values—to ensure that no data quality issues are being overlooked.

In [ ]:
# Select only numeric columns
numeric_data = data.select_dtypes(include=[np.number])

# Compute correlation matrix
corr_matrix = numeric_data.corr()

# Plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()

: 

In [ ]:
ordinal_features = ['Alcohol_Consumption', 'Obesity', 'Healthcare_Access', 
                    'Preventive_Care', 'Seafood_Consumption']

binary_features = ['Smoking_Status', 'Hepatitis_B_Status', 'Hepatitis_C_Status',
                   'Diabetes', 'Screening_Availability', 'Treatment_Availability', 
                   'Liver_Transplant_Access', 'Herbal_Medicine_Use']

nominal_features = ['Country', 'Region', 'Gender', 'Rural_or_Urban', 'Ethnicity']

numerical_features = ['Population', 'Incidence_Rate', 'Mortality_Rate', 'Age', 'Cost_of_Treatment', 'Survival_Rate']

target_variable = ['Prediction']


: 

In [ ]:
all_columns = set(data.columns)  # Convert dataset columns to a set

# Create a combined set for verification (without modifying lists)
defined_features = set(ordinal_features) | set(binary_features) | set(nominal_features) | set(target_variable) | set(numerical_features)

# Find missing or extra columns
missing_columns = all_columns - defined_features
extra_columns = defined_features - all_columns

if not missing_columns and not extra_columns:
    print("Defined features cover all columns!")
else:
    print("Check the following:")
    if missing_columns:
        print(f"Missing columns: {missing_columns}")
    if extra_columns:
        print(f"Extra columns that may not exist in data: {extra_columns}")

: 

#### Ordinal Encoding

In [ ]:
ordinal_mappings = [
    ['Low', 'Moderate', 'High'],
    ['Underweight', 'Normal', 'Overweight', 'Obese'],
    ['Poor', 'Moderate', 'Good'],
    ['Poor', 'Moderate', 'Good'],
    ['Low', 'Medium', 'High']
]

ordinal_encoder = OrdinalEncoder(categories=ordinal_mappings)

data[ordinal_features] = ordinal_encoder.fit_transform(data[ordinal_features])

: 

#### Binary Encoding


In [ ]:
for col in binary_features:
    data[col] = data[col].map({
        'No': 0, 'Yes': 1, 
        'Negative': 0, 'Positive': 1,
        'Non-Smoker': 0, 'Smoker': 1,
        'Not Available': 0, 'Available': 1,
        })

: 

#### One-Hot Encoding

In [ ]:
# Apply OneHotEncoder
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_nominals = one_hot_encoder.fit_transform(data[nominal_features])

: 

In [ ]:
# Convert to DataFrame
nominal_feature_names = one_hot_encoder.get_feature_names_out(nominal_features)
df_nominal = pd.DataFrame(encoded_nominals, columns=nominal_feature_names, index=data.index)

# Drop original nominal columns and concatenate encoded features
data = data.drop(columns=nominal_features).join(df_nominal)

: 

In [ ]:
data.select_dtypes(exclude=[np.number]).columns

: 

In [ ]:
# Convert target variable 'Prediction' (Yes/No) to binary format
data['Prediction'] = data['Prediction'].map({'No': 0, 'Yes': 1})

: 

In [ ]:
data.head()

: 

#### Scaling

In [ ]:
scaler = StandardScaler()

# Create a copy of the dataset to store the scaled values (optional)
data_scaled = data.copy()

data_scaled[numerical_features] = scaler.fit_transform(data[numerical_features])

: 

In [ ]:
data_scaled.head()

: 

In [ ]:
# Saving the feature engineering components for model_deployment
with open("ordinal_encoder.pkl", "wb") as f:
    pickle.dump(ordinal_encoder, f)

with open("one_hot_encoder.pkl", "wb") as f:
    pickle.dump(one_hot_encoder, f)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

# Save feature names (to ensure correct order during inference)
with open("feature_names.pkl", "wb") as f:
    pickle.dump(list(X_train.columns), f)

print("\n✅ Feature names saved successfully!")

: 

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

X = data_scaled.drop(columns=['Prediction'])  # Fe
y = data_scaled['Prediction']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Data Split Successful: X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")

# Train Logistic Regression
log_reg = LogisticRegression(max_iter=1000, class_weight='balanced')
log_reg.fit(X_train, y_train)

# Predictions
y_pred_log = log_reg.predict(X_test)
y_pred_proba_log = log_reg.predict_proba(X_test)[:, 1]

# Evaluate performance
accuracy_log = accuracy_score(y_test, y_pred_log)
roc_auc_log = roc_auc_score(y_test, y_pred_proba_log)

print("\n Logistic Regression Performance:")
print(f"Accuracy: {accuracy_log:.4f}")
print(f"ROC-AUC Score: {roc_auc_log:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred_log, zero_division=1))

# Confusion Matrix
plt.figure(figsize=(5, 4))
cm = confusion_matrix(y_test, y_pred_log)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No", "Yes"], yticklabels=["No", "Yes"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - Logistic Regression")
plt.show()

: 

### Random Forest Classifier 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)

# Predictions
y_pred_rf = rf.predict(X_test)
y_pred_proba_rf = rf.predict_proba(X_test)[:, 1]

# Evaluate performance
accuracy_rf = accuracy_score(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(y_test, y_pred_proba_rf)

print("\n Random Forest Performance:")
print(f"Accuracy: {accuracy_rf:.4f}")
print(f"ROC-AUC Score: {roc_auc_rf:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred_rf, zero_division=1))

# Confusion Matrix
plt.figure(figsize=(5, 4))
cm = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=["No", "Yes"], yticklabels=["No", "Yes"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - Random Forest")
plt.show()

: 

### Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Train Gradient Boosting
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb.fit(X_train, y_train)

# Predictions
y_pred_gb = gb.predict(X_test)
y_pred_proba_gb = gb.predict_proba(X_test)[:, 1]

# Evaluate performance
accuracy_gb = accuracy_score(y_test, y_pred_gb)
roc_auc_gb = roc_auc_score(y_test, y_pred_proba_gb)

print("\n📌 Gradient Boosting Performance:")
print(f"Accuracy: {accuracy_gb:.4f}")
print(f"ROC-AUC Score: {roc_auc_gb:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred_gb, zero_division=1))

# Confusion Matrix
plt.figure(figsize=(5, 4))
cm = confusion_matrix(y_test, y_pred_gb)
sns.heatmap(cm, annot=True, fmt="d", cmap="Oranges", xticklabels=["No", "Yes"], yticklabels=["No", "Yes"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - Gradient Boosting")
plt.show()

: 

## Simple Deep learning Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Convert to NumPy arrays
X_train_nn = np.array(X_train)
X_test_nn = np.array(X_test)
y_train_nn = np.array(y_train)
y_test_nn = np.array(y_test)

# Define the Neural Network Architecture
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.3),  # Dropout to prevent overfitting
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_nn, y_train_nn, epochs=20, batch_size=32, validation_data=(X_test_nn, y_test_nn), verbose=1)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test_nn, y_test_nn)
print(f"\n Neural Network Performance:")
print(f"Test Accuracy: {test_acc:.4f}")

: 

In [ ]:
# Save ML model (Random Forest as an example)
with open("rf_model.pkl", "wb") as model_file:
    pickle.dump(rf, model_file)

# Save Deep Learning Model
model.save("deep_learning_model.keras")

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 